In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from geoid.tiger import Block

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

`sandiegodata.org-pedestrian-blocks-1` Last Update: 2019-10-08T19:01:57

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [ped_blocks](data/ped_blocks.csv)**. 
* ** [blocks](data/blocks.csv)**. 

## References
<ul><li> <strong>pedestrians</string>, <em>metapack+http://library.metatab.org/cityiq.io-pedestrians-san_diego-1.zip#pedestrians</em>. </li><li> <strong>walkways</string>, <em>metapack+http://library.metatab.org/cityiq.io-objects-san_diego-5.zip#walkways</em>. </li><li> <strong>blocks</string>, <em>shape+https://www2.census.gov/geo/pvs/tiger2010st/06_California/06073/tl_2010_06073_tabblock10.zip</em>. 2010 Census blocks for San Diego County</li><ul>

In [3]:
blocks = pkg.reference('blocks').geoframe().to_crs({'init' :'epsg:4326'})
blocks['geoid'] = blocks.GEOID10.apply(lambda v: str(Block.parse(v).as_acs() ))
blocks = blocks[['geoid','geometry']]

In [4]:
ped = pkg.reference('pedestrians').dataframe()
walkways = pkg.reference('walkways').geoframe()

In [5]:
def tod(v):
    if 6<=v.hour<11:
        return 'morning'
    elif 11<=v.hour<13:
        return 'lunch'
    elif 13<=v.hour<17:
        return 'afternoon'
    elif 17<=v.hour<22:
        return 'evening'
    else:
        return 'night'
    
ped['tod'] = ped.timestamp.apply(tod)
ped['dow'] = ped.timestamp.dt.dayofweek

ped.head()

,timestamp,locationuid,direction,speed,count,tod,dow
0,2018-11-01 08:00:00,004361eb,165,2.641,1,morning,3
1,2018-11-01 09:15:00,004361eb,165,2.641,1,morning,3
2,2018-11-01 11:45:00,004361eb,165,2.641,1,lunch,3
3,2018-11-01 11:45:00,004361eb,345,2.641,1,lunch,3
4,2018-11-01 12:45:00,004361eb,165,2.641,1,lunch,3


In [6]:
ped_tod = ped.groupby(['dow', 'tod','locationuid']).sum()['count'].to_frame().reset_index()
ped_tod.rename(columns={'count':'ped_count'}, inplace=True)

In [7]:
hours_map = {
    'morning': 11 - 6,
    'lunch': 13 - 11,
    'afternoon': 17 -13,
    'evening': 22 - 16,
    'night': (6+24) - 22
}
ped_tod['tod_hours'] = ped_tod.tod.apply(lambda v: hours_map[v])
ped_tod['density'] = ped_tod.ped_count / ped_tod.tod_hours



ped_tod.head()

,dow,tod,locationuid,ped_count,tod_hours,density
0,0,afternoon,004361eb,380,4,95.00
1,0,afternoon,0051796c,1899,4,474.75
2,0,afternoon,005f90ed,524,4,131.00
3,0,afternoon,006bbb6e,706,4,176.50
4,0,afternoon,006da86e,7663,4,1915.75


In [10]:
j = walkways.merge(ped_tod,on='locationuid')
j['geometry'] = j.centroid 
j = gpd.sjoin(j, blocks)
ped_blocks = j.groupby(['geoid','dow','tod']).density.mean().to_frame().reset_index()
ped_blocks['z']  = (ped_blocks.density - ped_tod.density.mean())/ped_tod.density.std(ddof=0)
ped_blocks['rank'] =  ped_blocks.density.rank(pct=True)
ped_blocks.head()

,geoid,dow,tod,density,z,rank
0,10100US060730002011063,0,afternoon,171.000000,-0.177390,0.693204
1,10100US060730002011063,0,evening,254.333333,-0.008872,0.772047
2,10100US060730002011063,0,lunch,10.500000,-0.501955,0.158206
3,10100US060730002011063,0,morning,19.800000,-0.483149,0.244460
4,10100US060730002011063,0,night,172.625000,-0.174104,0.695009


In [11]:
len(ped_blocks)

42375